In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.externals.six import StringIO   
from sklearn.tree import export_graphviz
import pydotplus
from IPython.display import Image

## 心臟病資料
    此資料主要在研究身體數值如何影響心臟病患者

In [2]:
df = pd.read_csv('heart.csv',encoding = 'utf8')
df.head()


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [3]:
import statsmodels.formula.api as smf
import pandas as pd
def forward_selected(data, response):
    remaining = set(data.columns)
    remaining.remove(response)
    selected = []
    current_score, best_new_score = 0.0, 0.0
    while remaining and current_score == best_new_score:
        scores_with_candidates = []
        for candidate in remaining:
            formula = "{} ~ {} + 1".format(response,
                                           ' + '.join(selected + [candidate]))
            score = smf.ols(formula, data).fit().rsquared_adj
            scores_with_candidates.append((score, candidate))
        scores_with_candidates.sort()
        best_new_score, best_candidate = scores_with_candidates.pop()
        if current_score < best_new_score:
            remaining.remove(best_candidate)
            selected.append(best_candidate)
            current_score = best_new_score
    formula = "{} ~ {} + 1".format(response,
                                   ' + '.join(selected))
    model = smf.ols(formula, data).fit()
    return model

df = pd.read_csv("heart.csv")
df = pd.DataFrame(df).ix[:,1:]

model = forward_selected(df, 'target')
print(model.model.formula)
print(model.summary())

C:\Users\asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


target ~ exang + ca + oldpeak + cp + sex + thal + thalach + trestbps + slope + restecg + 1
                            OLS Regression Results                            
Dep. Variable:                 target   R-squared:                       0.516
Model:                            OLS   Adj. R-squared:                  0.499
Method:                 Least Squares   F-statistic:                     31.12
Date:                Wed, 17 Apr 2019   Prob (F-statistic):           1.47e-40
Time:                        20:46:41   Log-Likelihood:                -108.80
No. Observations:                 303   AIC:                             239.6
Df Residuals:                     292   BIC:                             280.4
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------

## Feacture Selection
    利用Forward selection方式選擇variable
    而上面選出了10個參數，不過為了decision tree 的大小
    我選用了4個參數 分別為sex，cp，ca，thal
    sex為性別
    cp為是否之前有胸口疼痛
    ca為number of major vessels (0-3) colored by flourosopy
    thal為3 = normal; 6 = fixed defect; 7 = reversable defect
  

In [4]:
#df = df.drop('age', axis = 1)
df = df.drop('exang', axis = 1)
df = df.drop('oldpeak', axis = 1)
df = df.drop('thalach', axis = 1)
df = df.drop('trestbps', axis = 1)
df = df.drop('slope', axis = 1)
df = df.drop('restecg', axis = 1)
df = df.drop('fbs', axis = 1)
df = df.drop('chol', axis = 1)

將沒有要用的參數刪除

In [5]:
df['target'].value_counts()

1    165
0    138
Name: target, dtype: int64

In [6]:
df.isnull().sum()

sex       0
cp        0
ca        0
thal      0
target    0
dtype: int64

In [7]:
from sklearn.model_selection import train_test_split


In [8]:
df_train, df_test = train_test_split(df, test_size=0.2)

In [9]:
y = df_train['target'].values
df_train = df_train.drop('target', 1)
y

array([0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0,
       0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1,
       0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1,
       1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1,
       1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1,
       0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0],
      dtype=int64)

In [12]:
dtree=DecisionTreeClassifier()
dtree.fit(df_train,y)

dot_data = StringIO()
export_graphviz(dtree, 
                out_file=dot_data,  
                filled=True, 
                feature_names=list(df_train),
                class_names=['not health','health'],
                special_characters=True)

graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
graph.write_pdf("tree.pdf")

True

建出decision tree來判斷是否擁有心臟疾病

In [13]:
dtree.feature_importances_

array([0.05424815, 0.26317341, 0.22056292, 0.46201552])

In [14]:
y_test = df_test['target'].values
X_test = df_test.drop('target', 1)

y_predict = dtree.predict(X_test)

y_predict

array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0,
       1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1,
       1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1], dtype=int64)

In [15]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_predict)

0.7868852459016393

得出的精確度為0.78688